In [1]:
# Utils packages
import ast
import itertools
from math import ceil
from pathlib import Path, PurePath, PurePosixPath
from typing import Union

# Data collector packages
from hentai import Utils, Hentai, Option, Sort

# Analysis packages
import pandas as pd
from PIL import Image
from tqdm import tqdm
import apache_beam as beam

# Compute packages
import tensorflow as tf
import tensorflow.experimental.numpy as np

## Data collection

We will use the information and media collected on the nhentai.net website. Recording each entry in a 'metadata.csv' file that aggregates information about the hentai, including links to download pages.

In [2]:
N_SAMPLE = 20 # Number of sample to refetch from hentai.
DATA_PATH = "data" # Directory to drop collected data.
SEARCH_QUERY = "tag:uncensored" # nhentai search query.
HENTAI_IDS = [] # Manual Hentai IDs to download.
COLLECT_MODE = "manual" # Sampling mode.

In [3]:
def get_hentai_info(hentai: Hentai):
    return hentai.dictionary(Option.all())


def get_random_hentai():
    hentai = Utils.get_random_hentai()
    hentai = get_hentai_info(hentai)
    return hentai


def search_hentai(query: str, page: int):
    return Utils.search_by_query(
        query,
        page,
        sort=Sort.PopularMonth
    )

In [4]:
# Query the nhentai.net API to get N number of random hentai
if COLLECT_MODE == "search":
    collected_hentais_df = pd.DataFrame([
        pd.Series(get_hentai_info(h))
        for hs in [
            search_hentai(SEARCH_QUERY, p)
            for p in range(1, ceil(N_SAMPLE / 25) + 1)
        ]
        for h in hs
    ])
elif COLLECT_MODE == "random":
    collected_hentais_df = pd.DataFrame([
        pd.Series(get_random_hentai())
        for _ in range(N_SAMPLE)
    ])
else:
    collected_hentais_df = pd.DataFrame([
        pd.Series(get_hentai_info(Hentai(h)))
        for h in HENTAI_IDS
    ])
collected_hentais_df

""


## Read dataset file

In [5]:
# The metadata NDJSON file that will contain the raw information about the hentai
metadata_path = PurePath(DATA_PATH) / "raw" / "metadata.ndjson"

In [6]:
hentais_df = pd.read_json(
    metadata_path,
    orient="records",
    lines=True,
    encoding="utf8"
)
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages
0,264920,Gakuseikaishitai,,https://nhentai.net/g/264920,https://nhentai.net/api/gallery/264920,1375429,1551668645,18469,"[sole male, ahegao, anal, big breasts, nakadas...",[],[kaguya-sama wa kokurasetai],"[chika fujiwara, miyuki shirogane, kaguya shin...",[english],[ginhaha],[doujinshi],https://t.nhentai.net/galleries/1375429/cover.jpg,https://t.nhentai.net/galleries/1375429/thumb.jpg,[https://i.nhentai.net/galleries/1375429/1.jpg...,23
1,352188,Boku wa Chiisana Succubus no Shimobe - I am a ...,,https://nhentai.net/g/352188,https://nhentai.net/api/gallery/352188,1873098,1616489585,5016,"[wings, garter belt, group, uncensored, story ...",[],[],[],"[english, translated]",[tamano kedama],[manga],https://t.nhentai.net/galleries/1873098/cover.jpg,https://t.nhentai.net/galleries/1873098/thumb.jpg,[https://i.nhentai.net/galleries/1873098/1.jpg...,195
2,339950,Papakatsu de Deatta Ko ga Inma kamo Shirenai,,https://nhentai.net/g/339950,https://nhentai.net/api/gallery/339950,1801880,1608241831,10139,"[sole male, beauty mark, lolicon, mouth mask, ...",[kedama gyuunyuu],[original],[],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1801880/cover.png,https://t.nhentai.net/galleries/1801880/thumb.png,[https://i.nhentai.net/galleries/1801880/1.png...,15
3,295471,Lily Spice,,https://nhentai.net/g/295471,https://nhentai.net/api/gallery/295471,1543338,1577761543,5456,"[sole male, dark skin, demon girl, gloves, lac...",[kedama gyuunyuu],[original],[],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1543338/cover.png,https://t.nhentai.net/galleries/1543338/thumb.png,[https://i.nhentai.net/galleries/1543338/1.png...,30
4,248327,Eld Marriage,,https://nhentai.net/g/248327,https://nhentai.net/api/gallery/248327,1293671,1538215810,3716,"[sole male, lolicon, sole female, stockings, t...",[kedama gyuunyuu],[azur lane],[eldridge],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1293671/cover.jpg,https://t.nhentai.net/galleries/1293671/thumb.jpg,[https://i.nhentai.net/galleries/1293671/1.jpg...,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],[mai-otome],"[shizuru fujino, erstin ho, nina wang, yohko h...",[japanese],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18
3748,243825,Count2.4 CGF Soushuuhen,,https://nhentai.net/g/243825,https://nhentai.net/api/gallery/243825,1274983,1534955886,1167,"[bondage, anal, gag, nakadashi, stockings]",[count2.4],[the idolmaster],"[koume shirasaka, mayu sakuma, mika jougasaki,...",[japanese],[nishi],[doujinshi],https://t.nhentai.net/galleries/1274983/cover.jpg,https://t.nhentai.net/galleries/1274983/thumb.jpg,[https://i.nhentai.net/galleries/1274983/1.jpg...,140
3749,230693,ふたりきり,,https://nhentai.net/g/230693,https://nhentai.net/api/gallery/230693,1213137,1524110819,70,"[anal, dark skin, males only, shotacon, yaoi, ...",[],[dragon ball z],"[son goku, turles]",[japanese],[],[doujinshi],https://t.nhentai.net/galleries/1213137/cover.png,https://t.nhentai.net/galleries/1213137/thumb.png,[https://i.nhentai.net/galleries/1213137/1.png...,15
3750,115897,Otoko no Tatakai 18,,https://nhentai.net/g/115897,https://nhentai.net/api/gallery/115897,736813,1410126146,49,"[fundoshi, humiliation, full censorship, femdo...",[ito ichizo shouten],[],[],[japanese],[ito ichizo],[doujinshi],https://t.nhentai.net/galleries/736813/cover.jpg,https://t.nhentai.net/galleries/736813/thumb.jpg,"[https://i.nhentai.net/galleries/736813/1.jpg,...",17


## Save collected dataset

In [7]:
hentais_df = pd.concat([collected_hentais_df, hentais_df], ignore_index=True)
hentais_df = hentais_df.drop_duplicates("id")
hentais_df.to_json(
    metadata_path,
    orient="records",
    lines=True
)

## Download images

In [8]:
def download_hentai(hentai: pd.Series):
    # A hentai will be stored based on the gallery ID instead of the medai ID
    hentai_path = Path(DATA_PATH) / "raw" / str(hentai.id)
    # Since downloading a hentai is an expensive operation, we only
    # download when the hentai ID directory does not exist
    if not hentai_path.is_dir():
        try:
            hentai = Hentai(hentai.id)
            hentai.download(folder=hentai_path, progressbar=True)
            return True
        except Exception:
            pass
    return False

In [9]:
downloaded_hentais_df = hentais_df.apply(download_hentai, axis=1)

In [10]:
print("Number of downloaded hentais : {}".format(downloaded_hentais_df.sum()))

Number of downloaded hentais : 0


## Remove corrupted images

In [11]:
def hentai_filename(hentai: pd.Series):
    return [
        str(PurePosixPath(str(hentai["id"])) / PurePath(url).name)
        for url in hentai["image_urls"]
    ]

In [12]:
hentais_df = hentais_df.assign(
    filename=hentais_df.apply(hentai_filename, axis=1)
)
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages,filename
0,264920,Gakuseikaishitai,,https://nhentai.net/g/264920,https://nhentai.net/api/gallery/264920,1375429,1551668645,18469,"[sole male, ahegao, anal, big breasts, nakadas...",[],[kaguya-sama wa kokurasetai],"[chika fujiwara, miyuki shirogane, kaguya shin...",[english],[ginhaha],[doujinshi],https://t.nhentai.net/galleries/1375429/cover.jpg,https://t.nhentai.net/galleries/1375429/thumb.jpg,[https://i.nhentai.net/galleries/1375429/1.jpg...,23,"[264920/1.jpg, 264920/2.jpg, 264920/3.jpg, 264..."
1,352188,Boku wa Chiisana Succubus no Shimobe - I am a ...,,https://nhentai.net/g/352188,https://nhentai.net/api/gallery/352188,1873098,1616489585,5016,"[wings, garter belt, group, uncensored, story ...",[],[],[],"[english, translated]",[tamano kedama],[manga],https://t.nhentai.net/galleries/1873098/cover.jpg,https://t.nhentai.net/galleries/1873098/thumb.jpg,[https://i.nhentai.net/galleries/1873098/1.jpg...,195,"[352188/1.jpg, 352188/2.jpg, 352188/3.jpg, 352..."
2,339950,Papakatsu de Deatta Ko ga Inma kamo Shirenai,,https://nhentai.net/g/339950,https://nhentai.net/api/gallery/339950,1801880,1608241831,10139,"[sole male, beauty mark, lolicon, mouth mask, ...",[kedama gyuunyuu],[original],[],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1801880/cover.png,https://t.nhentai.net/galleries/1801880/thumb.png,[https://i.nhentai.net/galleries/1801880/1.png...,15,"[339950/1.png, 339950/2.png, 339950/3.png, 339..."
3,295471,Lily Spice,,https://nhentai.net/g/295471,https://nhentai.net/api/gallery/295471,1543338,1577761543,5456,"[sole male, dark skin, demon girl, gloves, lac...",[kedama gyuunyuu],[original],[],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1543338/cover.png,https://t.nhentai.net/galleries/1543338/thumb.png,[https://i.nhentai.net/galleries/1543338/1.png...,30,"[295471/1.png, 295471/2.png, 295471/3.png, 295..."
4,248327,Eld Marriage,,https://nhentai.net/g/248327,https://nhentai.net/api/gallery/248327,1293671,1538215810,3716,"[sole male, lolicon, sole female, stockings, t...",[kedama gyuunyuu],[azur lane],[eldridge],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1293671/cover.jpg,https://t.nhentai.net/galleries/1293671/thumb.jpg,[https://i.nhentai.net/galleries/1293671/1.jpg...,22,"[248327/1.jpg, 248327/2.jpg, 248327/3.jpg, 248..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],[mai-otome],"[shizuru fujino, erstin ho, nina wang, yohko h...",[japanese],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18,"[19145/1.jpg, 19145/2.jpg, 19145/3.jpg, 19145/..."
3748,243825,Count2.4 CGF Soushuuhen,,https://nhentai.net/g/243825,https://nhentai.net/api/gallery/243825,1274983,1534955886,1167,"[bondage, anal, gag, nakadashi, stockings]",[count2.4],[the idolmaster],"[koume shirasaka, mayu sakuma, mika jougasaki,...",[japanese],[nishi],[doujinshi],https://t.nhentai.net/galleries/1274983/cover.jpg,https://t.nhentai.net/galleries/1274983/thumb.jpg,[https://i.nhentai.net/galleries/1274983/1.jpg...,140,"[243825/1.jpg, 243825/2.jpg, 243825/3.jpg, 243..."
3749,230693,ふたりきり,,https://nhentai.net/g/230693,https://nhentai.net/api/gallery/230693,1213137,1524110819,70,"[anal, dark skin, males only, shotacon, yaoi, ...",[],[dragon ball z],"[son goku, turles]",[japanese],[],[doujinshi],https://t.nhentai.net/galleries/1213137/cover.png,https://t.nhentai.net/galleries/1213137/thumb.png,[https://i.nhentai.net/galleries/1213137/1.png...,15,"[230693/1.png, 230693/2.png, 230693/3.png, 230

We will filter every hentais that contain at least one corrupted images, since pages have order dependencies.

In [13]:
def preprocess_image(filename: str):
    filename = Path(filename)
    preproc_filename = filename.parent / Path(filename.stem + ".jpg")
    image_preproc_dir = Path(DATA_PATH) / "preprocessed"
    image_preproc_path = image_preproc_dir / preproc_filename
    # Preprocess only those images that do not exist in
    # the destination dataset for performance reason
    if not image_preproc_path.is_file():
        try:
            image = Image.open(Path(DATA_PATH) / "raw" / filename)
            # Normalize images to be JPEG compatible
            image = image.convert("RGB")
            image_preproc_dir.mkdir(parents=True, exist_ok=True)
            image.save(image_preproc_path, "JPEG")
        except Exception:
            return None
    return str(PurePath(preproc_filename))


def preprocess_images(hentai: pd.Series):
    return [
        preprocess_image(filename)
        for filename in hentai["filename"]
    ]

In [14]:
tqdm.pandas()
hentais_df = hentais_df.assign(
    filename=hentais_df.progress_apply(preprocess_images, axis=1)
)

100%|██████████| 3752/3752 [00:25<00:00, 150.01it/s]


In [15]:
hentais_df = hentais_df[hentais_df["filename"].apply(all)]
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages,filename
0,264920,Gakuseikaishitai,,https://nhentai.net/g/264920,https://nhentai.net/api/gallery/264920,1375429,1551668645,18469,"[sole male, ahegao, anal, big breasts, nakadas...",[],[kaguya-sama wa kokurasetai],"[chika fujiwara, miyuki shirogane, kaguya shin...",[english],[ginhaha],[doujinshi],https://t.nhentai.net/galleries/1375429/cover.jpg,https://t.nhentai.net/galleries/1375429/thumb.jpg,[https://i.nhentai.net/galleries/1375429/1.jpg...,23,"[264920\1.jpg, 264920\2.jpg, 264920\3.jpg, 264..."
1,352188,Boku wa Chiisana Succubus no Shimobe - I am a ...,,https://nhentai.net/g/352188,https://nhentai.net/api/gallery/352188,1873098,1616489585,5016,"[wings, garter belt, group, uncensored, story ...",[],[],[],"[english, translated]",[tamano kedama],[manga],https://t.nhentai.net/galleries/1873098/cover.jpg,https://t.nhentai.net/galleries/1873098/thumb.jpg,[https://i.nhentai.net/galleries/1873098/1.jpg...,195,"[352188\1.jpg, 352188\2.jpg, 352188\3.jpg, 352..."
2,339950,Papakatsu de Deatta Ko ga Inma kamo Shirenai,,https://nhentai.net/g/339950,https://nhentai.net/api/gallery/339950,1801880,1608241831,10139,"[sole male, beauty mark, lolicon, mouth mask, ...",[kedama gyuunyuu],[original],[],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1801880/cover.png,https://t.nhentai.net/galleries/1801880/thumb.png,[https://i.nhentai.net/galleries/1801880/1.png...,15,"[339950\1.jpg, 339950\2.jpg, 339950\3.jpg, 339..."
3,295471,Lily Spice,,https://nhentai.net/g/295471,https://nhentai.net/api/gallery/295471,1543338,1577761543,5456,"[sole male, dark skin, demon girl, gloves, lac...",[kedama gyuunyuu],[original],[],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1543338/cover.png,https://t.nhentai.net/galleries/1543338/thumb.png,[https://i.nhentai.net/galleries/1543338/1.png...,30,"[295471\1.jpg, 295471\2.jpg, 295471\3.jpg, 295..."
4,248327,Eld Marriage,,https://nhentai.net/g/248327,https://nhentai.net/api/gallery/248327,1293671,1538215810,3716,"[sole male, lolicon, sole female, stockings, t...",[kedama gyuunyuu],[azur lane],[eldridge],"[english, translated]",[tamano kedama],[doujinshi],https://t.nhentai.net/galleries/1293671/cover.jpg,https://t.nhentai.net/galleries/1293671/thumb.jpg,[https://i.nhentai.net/galleries/1293671/1.jpg...,22,"[248327\1.jpg, 248327\2.jpg, 248327\3.jpg, 248..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],[mai-otome],"[shizuru fujino, erstin ho, nina wang, yohko h...",[japanese],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18,"[19145\1.jpg, 19145\2.jpg, 19145\3.jpg, 19145\..."
3748,243825,Count2.4 CGF Soushuuhen,,https://nhentai.net/g/243825,https://nhentai.net/api/gallery/243825,1274983,1534955886,1167,"[bondage, anal, gag, nakadashi, stockings]",[count2.4],[the idolmaster],"[koume shirasaka, mayu sakuma, mika jougasaki,...",[japanese],[nishi],[doujinshi],https://t.nhentai.net/galleries/1274983/cover.jpg,https://t.nhentai.net/galleries/1274983/thumb.jpg,[https://i.nhentai.net/galleries/1274983/1.jpg...,140,"[243825\1.jpg, 243825\2.jpg, 243825\3.jpg, 243..."
3749,230693,ふたりきり,,https://nhentai.net/g/230693,https://nhentai.net/api/gallery/230693,1213137,1524110819,70,"[anal, dark skin, males only, shotacon, yaoi, ...",[],[dragon ball z],"[son goku, turles]",[japanese],[],[doujinshi],https://t.nhentai.net/galleries/1213137/cover.png,https://t.nhentai.net/galleries/1213137/thumb.png,[https://i.nhentai.net/galleries/1213137/1.png...,15,"[230693\1.jpg, 230693\2.jpg, 230693\3.jpg, 230

In [16]:
# The metadata CSV file that will contain the raw information about the hentai
metadata_path = PurePath(DATA_PATH) / "preprocessed" / "metadata.ndjson"
hentais_df = hentais_df[[
    "id",
    "title",
    "num_favorites",
    "tag",
    "group",
    "parody",
    "character",
    "language",
    "artist",
    "category",
    "num_pages",
    "filename"
]]
hentais_df.to_json(
    metadata_path,
    orient="records",
    lines=True
)
print("Number of preprocessed images :", len(hentais_df))

Number of preprocessed images : 3744
